# Hardware Compression Cost Report

## Version A: Cost ceiling $800 ("Functional Shrink")
- Goal: make the current Raspberry Pi 5 + camera + AI hat setup physically smaller and cleaner without redesigning a brand new computer.
- Strategy: keep an off‑the‑shelf compute module (e.g. Raspberry Pi Compute Module / Jetson Orin Nano module / Pi 5 equivalent brain) and design a **custom carrier board** that:
  - removes unused ports (extra USB, HDMI, GPIO headers, etc.)
  - keeps only camera CSI connector, power in, 1x USB for setup, cooling header, and AI accelerator header
  - fits in a compact 3D‑printed enclosure.
- What the vendor actually does:
  1. electrical layout for a 2‑ or 4‑layer carrier PCB
  2. basic enclosure model (3D printed or PLA/ABS block)
  3. assemble 1 prototype
- Not included:
  - FCC / EMI validation
  - metal enclosure thermal tuning
  - production‑ready documentation
  - full rewrite of firmware/drivers
- Why it can hit ~$600‑$900 total:
  - You are NOT asking them to redesign the CPU, RAM, DDR traces, PCIe, etc. You are saying: “Treat the compute module like a black box, just build us a minimal breakout board around it so the whole unit is smaller and only has what we need.”
- When to ask for this quote:
  - early demo for investors / pilot in one restaurant
  - you only need 1–2 working boxes, not mass production.

## Version B: Professional Product Version ($3,000 – $8,000 prototype; $10,000+ for production readiness)
- Goal: turn the prototype into a real product SKU that is manufacturable, reliable, thermally stable, and documented.
- Strategy:
  - fully custom PCB with integrated compute, AI accelerator, camera interface, storage, power regulation
  - enclosure designed for durability (machined aluminum or injection‑mold‑ready geometry)
  - thermal solution (fan or passive sink) validated for 24/7 uptime in a hot kitchen
  - bring‑up of Linux image and drivers so it boots into your app.
- What the vendor delivers:
  1. Schematics + PCB (Gerbers, BOM)
  2. Mechanical CAD (STEP)
  3. 1–2 assembled prototypes
  4. Power + thermal test notes
  5. Roadmap for EMC/FCC compliance
- Why the cost jumps:
  - High‑speed layout around CPU and AI accelerator is nontrivial, and they own some electrical risk.
  - You are asking them to create *your* hardware platform, not a Raspberry Pi anymore.
- When to ask for this quote:
  - you want something you can put in front of restaurant managers or early customers as "our device"
  - you are thinking about 10+ units and future production.

---
Below is the step‑by‑step setup guide for flashing Raspberry Pi OS, enabling the Arducam camera, and streaming the live video feed to a Mac.


# Raspberry Pi + Arducam IMX708 Setup Guide
This notebook gives step‑by‑step commands to configure a Raspberry Pi running Raspberry Pi OS (Trixie ARM64) to stream its camera feed to your Mac.

Use it as a reference: run these commands on your Pi or Mac terminal, not in Colab itself.

## 1. Flash Raspberry Pi OS
1. Download the image: `2025-10-01-raspios-trixie-arm64.img`
2. Open Raspberry Pi Imager on Mac → Use Custom → select that `.img`
3. Click the gear icon and enable:
   - SSH
   - Set username/password (e.g. `pi` / `raspberry`)
   - Set Wi‑Fi SSID + password + country code
4. Choose storage (micro‑SD) → Write
5. Insert the card into the Pi and power it on.

## 2. Connect from Mac to Pi
```bash
# Find the Pi's IP address
ping raspberrypi.local

# SSH into it
ssh pi@raspberrypi.local
# or use its IP directly, e.g.
ssh pi@192.168.1.42
```
If you get "Permission denied," double‑check the username and password.

## 3. Enable Camera and SSH (if not already)
```bash
sudo raspi-config
# Interface Options → Camera → Enable
# Interface Options → SSH → Enable
sudo reboot
```

## 4. Verify Camera Detection
After reboot:
```bash
libcamera-hello --list-cameras
```
Expected output includes `imx708`. If not, reseat the ribbon cable (blue tab faces USB/Ethernet ports).

## 5. Install XQuartz on Mac
1. Go to https://www.xquartz.org
2. Install and reboot macOS
3. Open XQuartz and leave it running

## 6. View Camera on Mac via X Forwarding
On Mac terminal:
```bash
ssh -Y pi@raspberrypi.local
libcamera-hello -t 0
```
A live preview window should appear on your Mac through XQuartz.

## 7. Optional — Stream Over Network (VLC)
On Pi:
```bash
sudo apt update
sudo apt install -y ffmpeg
libcamera-vid -t 0 --inline --listen -o tcp://0.0.0.0:8554
```
Leave that running.

On Mac (VLC → File → Open Network):
```
tcp/h264://raspberrypi.local:8554
```
Now you get live video in VLC.

## 8. Common Fixes
- Command not found → `sudo apt install -y libcamera-apps`
- Black preview → wrong cable orientation
- No ping → recheck Wi‑Fi credentials in imager setup
- Slow preview → try the VLC network method

## 9. Complete
You can now view your 12 MP Arducam IMX708 camera feed directly on your Mac using either XQuartz or VLC.
